# ActBlue donor profile

In [34]:
import json
import numpy as np
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import os

In [35]:
%matplotlib inline

plt.style.use('ggplot')

pd.options.display.float_format = '{:,.2f}'.format

In [36]:
def read_or_save(name, func):
    path = 'pickles/' + name + '.pickle.gz'
    if (os.path.isfile(path)):
        return pd.read_pickle(path)
    else:
        result = func()
        os.makedirs('pickles', exist_ok=True)
        result.to_pickle(path)
        return result

In [37]:
committee_id = "C00401224"
year_since = 2013

### Connect to the PostgreSQL database

In [38]:
with open("config.json") as f:
    conf = json.load(f)
conn = psycopg2.connect(
    dbname=conf['dbname'],
    user=conf['user'],
    host=conf['host'],
    password=conf['password']
)

### Where do the donors giving through ActBlue come from geographically? Any striking or interesting patterns, or zip codes that typically don’t contribute?

In [39]:
actblue_states = read_or_save(
    'actblue_states',
    lambda: pd.read_sql("""
    SELECT contributor_state,
           count(*),
           sum(contribution_amount)
    FROM fec_contributions
    WHERE filing_id IN
        (SELECT filing_id
         FROM fec_pac_summaries
         JOIN fec_amended_filings USING (filing_id)
         WHERE filer_committee_id_number = '""" + committee_id + """'
           AND extract(YEAR
                       FROM coverage_through_date) > """ + str(year_since) + """
         ORDER BY coverage_through_date DESC)
      AND form_type = 'SA11AI'
    GROUP BY contributor_state
    """, con=conn)
)
actblue_states.sort_values(by=['sum'], ascending=False)

,contributor_state,count,sum
30,CA,12062776,"283,218,125.72"
44,NY,5628188,"157,311,026.41"
50,MA,2773802,"72,470,951.55"
51,TX,3112392,"65,044,446.34"
1,FL,3159095,"64,572,158.63"
11,IL,2471078,"54,557,368.54"
32,WA,2741416,"51,188,999.37"
52,PA,2163765,"44,781,490.78"
37,VA,1627316,"44,353,311.58"
74,MD,1467096,"40,147,179.65"


In [40]:
clinton_votes = pd.read_csv('clintonvotes.csv')

clinton_votes

votes_vs_contribs = actblue_states.merge(clinton_votes, left_on='contributor_state', right_on='postal', how='inner')

votes_vs_contribs['dollars_per'] = votes_vs_contribs['sum']/votes_vs_contribs['votes']

votes_vs_contribs.sort_values(by=['dollars_per'], ascending=False)

,contributor_state,count,sum,state,postal,votes,dollars_per
37,DC,473208,"29,613,824.08",District of Columbia,DC,282830,104.71
29,VT,500320,"9,498,225.91",Vermont,VT,178573,53.19
25,MT,314878,"7,379,330.20",Montana,MT,177709,41.52
26,NM,781732,"14,215,954.03",New Mexico,NM,385234,36.90
21,MA,2773802,"72,470,951.55",Massachusetts,MA,1995196,36.32
20,NY,5628188,"157,311,026.41",New York,NY,4556142,34.53
27,AK,199663,"3,906,274.67",Alaska,AK,116454,33.54
13,CA,12062776,"283,218,125.72",California,CA,8753792,32.35
47,NH,484771,"11,174,925.17",New Hampshire,NH,348526,32.06
33,WY,74931,"1,693,472.24",Wyoming,WY,55973,30.26


In [41]:
votes_vs_contribs['contribs_per'] = votes_vs_contribs['count']/votes_vs_contribs['votes']

votes_vs_contribs.sort_values(by=['contribs_per'], ascending=False)

,contributor_state,count,sum,state,postal,votes,dollars_per,contribs_per
29,VT,500320,"9,498,225.91",Vermont,VT,178573,53.19,2.80
26,NM,781732,"14,215,954.03",New Mexico,NM,385234,36.90,2.03
2,OR,1909208,"29,109,770.12",Oregon,OR,1002106,29.05,1.91
25,MT,314878,"7,379,330.20",Montana,MT,177709,41.52,1.77
27,AK,199663,"3,906,274.67",Alaska,AK,116454,33.54,1.71
37,DC,473208,"29,613,824.08",District of Columbia,DC,282830,104.71,1.67
15,WA,2741416,"51,188,999.37",Washington,WA,1742718,29.37,1.57
47,NH,484771,"11,174,925.17",New Hampshire,NH,348526,32.06,1.39
21,MA,2773802,"72,470,951.55",Massachusetts,MA,1995196,36.32,1.39
13,CA,12062776,"283,218,125.72",California,CA,8753792,32.35,1.38


### Questions to answer
> - Where do the donors giving through ActBlue come from geographically? Any striking or interesting patterns, or zip codes that typically don’t contribute?
> - Where is the money going? Are people giving to local candidates or are they giving to candidates around the country? Which campaigns/candidates/groups have done the best job tapping into this network?
> - What’s the gender breakdown and how does it compare to the gender breakdown of itemized contributions? Is it different?
> - Are these new donors?
> - Can we tell if these are recurring contributions or one time contributions?
> - Do the majority of these donors give repeatedly or are they one and done? How does that compare to itemized contributions? How “sticky” is this?


In [42]:
actblue_in_state = read_or_save(
    'actblue_in_state',
    lambda: pd.read_sql("""
    SELECT CASE
               WHEN beneficiary_candidate_state = contributor_state THEN TRUE
               ELSE FALSE
           END AS in_state,
           sum(contribution_amount),
           count(*)
    FROM fec_contributions
    JOIN fec_expenditures ON fec_contributions.filing_id IN
      (SELECT filing_id
       FROM fec_pac_summaries
       JOIN fec_amended_filings USING (filing_id)
       WHERE filer_committee_id_number = '""" + committee_id + """'
         AND extract(YEAR
                     FROM coverage_through_date) > 2016
       ORDER BY coverage_through_date DESC)
    AND fec_expenditures.filing_id = fec_contributions.filing_id
    AND fec_contributions.form_type = 'SA11AI'
    AND fec_expenditures.form_type = 'SB23'
    AND replace(transaction_id_number,'SB23_','') = replace(transaction_id,'SA11AI_','')
    AND beneficiary_candidate_state IS NOT NULL
    GROUP BY in_state
    """, con=conn)
)
actblue_in_state

,in_state,sum,count
0,False,"172,628,257.47",6124808
1,True,"112,972,802.28",2107032


In [43]:
actblue_freq = read_or_save(
    'actblue_freq',
    lambda: pd.read_sql("""
    SELECT COUNT,
           count(*) AS count_of_count
    FROM
      (SELECT contributor_first_name,
              contributor_last_name,
              left(contributor_zip_code,5),
              count(*) AS COUNT
       FROM fec_contributions
       WHERE filing_id IN
           (SELECT filing_id
            FROM fec_pac_summaries
            JOIN fec_amended_filings USING (filing_id)
            WHERE filer_committee_id_number = '""" + committee_id + """'
              AND extract(YEAR
                          FROM coverage_through_date) > 2016
            ORDER BY coverage_through_date DESC)
         AND form_type = 'SA11AI'
       GROUP BY contributor_first_name,
                contributor_last_name,
                left(contributor_zip_code,5)) AS donors
    GROUP BY COUNT
    """, con=conn)
)
actblue_freq

,count,count_of_count
0,251,67
1,1074,1
2,264,56
3,887,1
4,802,3
5,1350,1
6,1070,2
7,496,5
8,455,10
9,2265,1


In [44]:
actblue_addicts = read_or_save(
    'actblue_addicts',
    lambda: pd.read_sql("""
    SELECT COUNT,
           count(*) AS count_of_count,
           array_agg(DISTINCT contributor_organization_name),
           array_agg(DISTINCT contributor_first_name),
           array_agg(DISTINCT contributor_last_name),
           array_agg(DISTINCT contributor_zip_code)
    FROM
      (SELECT contributor_organization_name,
              contributor_first_name,
              contributor_last_name,
              left(contributor_zip_code,5) as contributor_zip_code,
              count(*) AS COUNT
       FROM fec_contributions
       WHERE filing_id IN
           (SELECT filing_id
            FROM fec_pac_summaries
            JOIN fec_amended_filings USING (filing_id)
            WHERE filer_committee_id_number = '""" + committee_id + """'
              AND extract(YEAR
                          FROM coverage_through_date) > 2016
            ORDER BY coverage_through_date DESC)
         AND form_type = 'SA11AI'
       GROUP BY contributor_organization_name,
                contributor_first_name,
                contributor_last_name,
                left(contributor_zip_code,5)) AS donors
    GROUP BY COUNT
    HAVING count(*) <= 3
    ORDER BY COUNT DESC
    LIMIT 30
    """, con=conn)
)
actblue_addicts



,count,count_of_count,array_agg,array_agg,array_agg,array_agg
0,11841,1,[None],[SIBYLLE],[BARLOW],[01742]
1,7900,1,[None],[TERRY],[WOLFE],[26508]
2,7108,1,[None],[VICKI],[FARRAR],[83714]
3,6807,1,[None],[WILLIAM],[CHEEK],[92115]
4,6518,1,[None],[RICHARD],[GOLDSTEIN],[11962]
5,6240,1,[None],[CAROLINE],[MERRIAM],[20007]
6,5726,1,[None],[JOHN],[COMELLA],[19103]
7,5512,1,[None],[LUCY],[HARMON],[75771]
8,5500,1,[None],[ROXANNE],[WARREN],[10025]
9,5341,1,[None],[ROBERT],[ECKEL],[18940]


### How candidates in the DCCC’s Red to Blue program do via ActBlue.

### Where do contributors give from?

### Whether candidates who pledge not to take corporate PAC money are getting more contributions than other candidates.

### More examples of ActBlue addicts.

### Spreadsheet of the 1,307 candidate committees that got at least some money through ActBlue this cycle.

### Unitemized contributions versus itemized for this point in the cycle for both the 2018 and 2014 cycle, and break it down by Republican versus Democrat?